In [9]:
# A star algorithm for searching 
# it is given a adj list & a heuristic function
# intro infos:
# list of nodes, lits of edges with weights, start node, end node
import heapq

class Cell:
    def __init__(self, i:int, j:int, val:int):
        # h -> heuristic, g -> graph_dist, f -> h+g
        self.h = 0
        self.g = float('inf')
        self.f = float('inf')
        self.parent: tuple[int, int] = (0, 0)
        self.pos = (i, j)
        self.val = val

class AStar:

    def __init__(self, rows: int, columns: int, grid: list[list[Cell]]):
        self.dim: tuple[int, int] = (rows, columns)
        self.grid = grid

    def cellInsideGrid(self, cell: Cell) -> bool:
        row, column = cell.pos
        R, C = self.dim
        return (row >= 0) and (row < R) and (column >= 0) and (column < C)

    def cellIsNotWall(self, cell:Cell) -> bool:
        return cell.val == 1

    def isDestination(self, cell: Cell, end: Cell) -> bool:
        i, j = cell.pos 
        desti, destj = end.pos 
        return (i == desti) and (j == destj)
    
    #compute the heuristik value of the cell( Euclidean distance to destination)
    def computeH(self, node1: Cell, node2: Cell) -> float:
        i1, j1 = node1.pos 
        i2, j2 = node2.pos 
        return ( (i1 - i2) ** 2 + (j1 - j2) ** 2) ** .5
    
    # Reconstruct the path
    def tracePath(self, end: Cell) -> None:
        print("The path is: ")
        path = []
        row, col = end.pos 
        endofpath = False
        # Trace the path from the destination to source using parent cells
        while not endofpath:
            parenti, parentj = self.grid[row][col].parent
            path.append((row, col))
            if parenti == row and parentj == col:
                endofpath = True
            row = parenti
            col = parentj

        # Add the source cell
        path.append((row, col))
        path.reverse()
        
        for node in path:
            print("-> ", node, end=" ")
        print()

    def astar(self, starNode: Cell, endNode: Cell) -> None:
        # Check if the source and destination are valid
        if not self.cellInsideGrid(starNode) or not self.cellInsideGrid(endNode):
            print(" Source node or end node are out of the grid")
            return 
        
        if not self.cellIsNotWall(starNode) or not self.cellIsNotWall(endNode):
            print("Either source or destination are walls")
            return 
        
        if self.isDestination(starNode, endNode):
            print(" The source is also the destination ")
            return 

        #Init the visited matrix
        visited = [ [ False for _ in range(self.dim[1]) ] for _ in range(self.dim[0])]
        # Init the startNode details 
        i, j = starNode.pos 
        self.grid[i][j].parent = (i, j)
        self.grid[i][j].f = 0
        self.grid[i][j].g = 0
        self.grid[i][j].h = 0

        priorq: list[tuple[float, int, int]] = []
        # used to solve the ordering after the f-value
        heapq.heappush(priorq, (0.0, i, j))
        founddest = False 
        while len(priorq) > 0:

            p = heapq.heappop(priorq)

            #Mark the cell as visited
            i, j = p[1], p[2]
            visited[i][j] = True 
            
            # Compute the neighbours 
            directions = [(0, 1), (0, -1), (1, 0), (-1, 0),
                        (1, 1), (1, -1), (-1, 1), (-1, -1)]

            for direc in directions:
                newi = i + direc[0]
                newj = j + direc[1]
                # Cell coresponding
                cell = Cell(newi, newj, -1)                
                # if the neighbour is not visited and valid
                if self.cellInsideGrid(cell) and self.cellIsNotWall(cell) and not visited[i][j]:
                    cell = self.grid[newi][newj]
                    # if the neighbour is the destination
                    if self.isDestination(cell, endNode):
                        # Set the parent coords
                        cell.parent = (i, j)
                        print(" The destination cell is found")
                        self.tracePath(endNode)
                        founddest = True
                        return
                    else:
                        # Compute the new distance
                        newg = self.grid[i][j] + 1.0
                        newh = self.computeH(cell, endNode)
                        newf = newg + newh

                        # Update the f-value if needed
                        if cell.f == float('inf') or cell.f > newf:
                            # add the cell with the new value if the priorityQ
                            heapq.heappush(priorq, (newf, newi, newj))
                            # update the cell infos
                            cell.f = newf 
                            cell.g = newg 
                            cell.h = newh 
                            cell.parent = (i, j)


        if not founddest:
            print("Failed to find the destination")




def main():
    grid = [
    [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 1],
    [1, 1, 1, 0, 1, 1, 0, 1, 0, 1],
    [0, 0, 1, 0, 1, 0, 0, 0, 0, 1],
    [1, 1, 1, 0, 1, 1, 1, 0, 1, 0],
    [1, 0, 1, 1, 1, 1, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 1, 0, 0, 1]
    ]

    newgrid = []
    for i in range(len(grid)):
        row = []
        for j in range(len(grid[i])):
            if grid[i][j] == 1:
                row.append( Cell(i, j, 1))
            else:
                row.append(Cell(i, j, 0))
        newgrid.append(row)
    
    A = AStar(9, 10, newgrid)
    startNode = newgrid[8][0]
    endNode = newgrid[0][0]
    A.astar(startNode, endNode)
main()

Failed to find the destination
